In [1]:
import keras
from keras.layers import *
from keras.models import Model
import numpy as np
import pandas as pd


Using TensorFlow backend.


In [2]:
filepath_train = './Data/train.csv'
train_data = pd.read_csv(filepath_train)
filepath_test = './Data/test.csv'
test_data = pd.read_csv(filepath_test)


In [3]:
train_data.shape

(42000, 785)

In [4]:
train_np_array = train_data.values

In [5]:
train_np_array

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [6]:
test_xy, train_xy = train_np_array[:8400, :], train_np_array[8400:, :]

In [7]:
test_xy.shape

(8400, 785)

In [8]:
train_xy.shape

(33600, 785)

In [9]:
X_train, Y_train = train_xy[:, 1:], train_xy[:,:1]
X_train.shape, Y_train.shape

((33600, 784), (33600, 1))

In [10]:
X_train = np.reshape(X_train, (-1,28,28,1))
X_train.shape

(33600, 28, 28, 1)

In [11]:
Y_train

array([[5],
       [0],
       [9],
       ...,
       [7],
       [6],
       [9]])

In [12]:
X_test, Y_test = test_xy[:, 1:], test_xy[:,:1]
X_test = np.reshape(X_test, (-1, 28, 28, 1))
X_test.shape, Y_test.shape

((8400, 28, 28, 1), (8400, 1))

In [13]:
Y_train = keras.utils.to_categorical(Y_train, 10)
Y_test = keras.utils.to_categorical(Y_test, 10)
X_train = X_train/255.
X_test = X_test/255

In [14]:
def createModel():
    inp = Input(X_train.shape[1:])
    x = Conv2D(32,kernel_size=[3,3],strides=[1,1],padding='SAME')(inp)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=[2,2],strides=[2,2])(x)
    x = Conv2D(64,kernel_size=[3,3],strides=[1,1],padding='SAME')(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=[2,2],strides=[2,2])(x)
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(10, activation = 'sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    return model

In [15]:
model = createModel()

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(x=X_train, y=Y_train, validation_split=0.1, epochs=10)

Train on 30240 samples, validate on 3360 samples
Epoch 1/10
30240/30240 [==============================] - 40s 1ms/step - loss: 0.8661 - acc: 0.7013 - val_loss: 0.1069 - val_acc: 0.9655
Epoch 2/10
30240/30240 [==============================] - 49s 2ms/step - loss: 0.3994 - acc: 0.8555 - val_loss: 0.0738 - val_acc: 0.9744
Epoch 3/10
30240/30240 [==============================] - 60s 2ms/step - loss: 0.3326 - acc: 0.8761 - val_loss: 0.0602 - val_acc: 0.9810
Epoch 4/10
30240/30240 [==============================] - 60s 2ms/step - loss: 0.2983 - acc: 0.8892 - val_loss: 0.0535 - val_acc: 0.9824
Epoch 5/10
30240/30240 [==============================] - 46s 2ms/step - loss: 0.2571 - acc: 0.9066 - val_loss: 0.0484 - val_acc: 0.9854
Epoch 6/10
30240/30240 [==============================] - 56s 2ms/step - loss: 0.2359 - acc: 0.9096 - val_loss: 0.0502 - val_acc: 0.9857
Epoch 7/10
30240/30240 [==============================] - 72s 2ms/step - loss: 0.2284 - acc: 0.9114 - val_loss: 0.0500 - val_acc:

In [19]:
model.save('./trained_model/digit_recog_92_98.h5')

In [20]:
model.evaluate(X_test, Y_test)

8400/8400 [==============================] - 3s 376us/step


[0.04946225354034964, 0.9855952380952381]

In [30]:
test_data = pd.read_csv('Data/test.csv')

In [31]:
test_data = test_data.values

In [32]:
test_data = test_data/255.
test_data = np.reshape(test_data, (-1, 28, 28, 1))
test_data.shape

(28000, 28, 28, 1)

In [33]:
predictions = model.predict(test_data)

In [34]:
predictions.shape

(28000, 10)

In [35]:
y = np.argmax(predictions, axis=1)

In [36]:
y.shape

(28000,)

In [37]:
y

array([2, 0, 9, ..., 3, 9, 2])

In [38]:
df = pd.DataFrame(y)

In [39]:
df = df.reset_index()
df.columns.values[0]='ImageId'
df['ImageId'] = df.index + 1

In [40]:
df.columns = ['ImageId', 'Label']
df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [41]:
df.to_csv('Data/mysub_cnn.csv', index=False, )